In [1]:
!pip install ultralytics

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/Desktop/SatellieteIMGDetection
import os
print(os.getcwd())



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection


In [ ]:
import json

# 하이퍼파라미터 튜닝
from ultralytics import YOLO
model = YOLO("yolo11x-obb.pt")
best_params = model.tune(
    data="data_combine.yaml",
    batch=4,
    epochs=15,
    iterations=50,
    optimizer="AdamW",
    plots=True,
    save=False,   # 모든 iteration에서 저장하지 않도록 설정
    val=True,
    lr0=0.001,
    scale=0.5,
    seed=44,
)

# 최적 하이퍼파라미터 저장
with open("best_params.json", "w") as f:
    json.dump(best_params, f)
print("Best parameters saved to best_params.json.")


Tuner: Initialized Tuner instance with 'tune_dir=runs\obb\tune3'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/50 with hyperparameters: {'lr0': 0.001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs\obb\tune3\tune_scatter_plots.png
Saved runs\obb\tune3\tune_fitness.png

Tuner: 1/50 iterations complete  (1911.71s)
Tuner: Results saved to runs\obb\tune3
Tuner: Best fitness=0.57245 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.79369, 'metrics/recall(B)': 0.68401, 'metrics/mAP50(B)': 0.73927, 'metrics/mAP50-95(B)': 0.55391, 'val/box_loss': 1.0375, 'val/cls_loss': 0.98109, '

In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11x-obb.pt")
results = model.train(
    data="data_combine.yaml",
    seed=44,
    epochs=20,
    # batch=3,
    # imgsz=1024,
    batch=4,
    imgsz=915,
    patience=15,
    device="0",
    optimizer = "AdamW",
    lr0= 0.00088,
    lrf= 0.01134,
    momentum= 0.82858,
    weight_decay= 0.0006,
    warmup_epochs= 2.33952,
    warmup_momentum= 0.58988,
    # box= 5.06927,
    box = 9.1,
    cls= 0.56241,
    dfl= 1.36767,
    hsv_h= 0.01707,
    hsv_s= 0.68688,
    hsv_v= 0.28102,
    degrees= 0.0,
    translate= 0.11263,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.45936,
    fliplr= 0.46892,
    bgr= 0.1,
    mosaic= 0.828,
    mixup= 0.0,
    copy_paste= 0.0,
    scale = 0.2,
    dropout = 0.3,
    )


New https://pypi.org/project/ultralytics/8.3.17 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_combine.yaml, epochs=20, time=None, patience=15, batch=4, imgsz=915, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=44, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

c:\Users\pook0\.conda\envs\UM\Lib\site-packages\torch\nn\modules\module.py:1160: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


AMP: checks passed 
WARNING  imgsz=[915] must be multiple of max stride 32, updating to [928]


train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 864 images, 140 backgrounds, 0 corrupt: 100%|██████████| 864/864 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      20/20      12.7G       1.17     0.9668      1.099        608        928: 100%|██████████| 216/216 [01:34<00:00,  2.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:04<00:00,  5.72it/s]

                   all         92       4910      0.828      0.698      0.767      0.591



20 epochs completed in 0.569 hours.
Optimizer stripped from runs\obb\train8\weights\last.pt, 118.4MB
Optimizer stripped from runs\obb\train8\weights\best.pt, 118.4MB

Validating runs\obb\train8\weights\best.pt...
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11x-obb summary (fused): 483 layers, 58,737,913 parameters, 0 gradients, 202.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:05<00:00,  4.05it/s]


                   all         92       4910      0.829      0.698      0.767      0.591
               ship(S)         79       4334      0.904      0.673       0.76      0.556
               ship(L)         61        576      0.754      0.722      0.774      0.625
Speed: 0.8ms preprocess, 28.1ms inference, 0.0ms loss, 7.4ms postprocess per image
Results saved to runs\obb\train8


# 승우 1000분 hyperparameter Tune

In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

base = "yolo11x-obb.pt"
# resume ="runs/obb/train13/weights/best.pt"
# YOLO 모델 초기화
model = YOLO(base) 
results = model.train(
    data="data_combine.yaml",
    name = "x_s768_bgr7538_scale084_",
    seed=44,
    epochs=30,
    batch=6,
    imgsz=768,
    patience=18,
    device="0",
    optimizer = "AdamW",
    lr0= 0.00333,
    lrf= 0.00305,
    momentum= 0.7,
    weight_decay= 0.00049,
    warmup_epochs= 4.78773,
    warmup_momentum= 0.86096,
    # box= 5.83147,
    box = 9.12,
    cls= 0.55571,
    dfl= 1.72141,
    hsv_h= 0.01527,
    hsv_s= 0.68368,
    hsv_v= 0.35707,
    degrees= 0.0,
    translate= 0.11826,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.65733,
    fliplr= 0.44244,
    bgr= 0.7538,
    mosaic= 0.9319,
    mixup= 0.0,
    copy_paste= 0.0,
    scale = 0.84,
    
    dropout = 0.35,
    # label_smoothing = 0.3,
    )

Ultralytics 8.3.17  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_combine.yaml, epochs=30, time=None, patience=18, batch=6, imgsz=768, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=x_s768_bgr7538_scale084_, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=44, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.35, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

c:\Users\pook0\.conda\envs\UM\Lib\site-packages\torch\nn\modules\module.py:1160: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


AMP: checks passed 


train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 864 images, 140 backgrounds, 0 corrupt: 100%|██████████| 864/864 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      30/30      13.2G      1.485      1.156      1.476        269        768: 100%|██████████| 144/144 [01:03<00:00,  2.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03<00:00,  5.18it/s]

                   all         92       4910      0.753      0.631      0.689      0.486



30 epochs completed in 0.595 hours.
Optimizer stripped from runs\obb\x_s768_bgr7538_scale084_\weights\last.pt, 118.3MB
Optimizer stripped from runs\obb\x_s768_bgr7538_scale084_\weights\best.pt, 118.3MB

Validating runs\obb\x_s768_bgr7538_scale084_\weights\best.pt...
Ultralytics 8.3.17  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11x-obb summary (fused): 483 layers, 58,737,913 parameters, 0 gradients, 202.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:05<00:00,  2.93it/s]


                   all         92       4910      0.756       0.63      0.689      0.486
               ship(S)         79       4334      0.896      0.572      0.695      0.478
               ship(L)         61        576      0.615      0.688      0.683      0.495
Speed: 0.7ms preprocess, 20.4ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to runs\obb\x_s768_bgr7538_scale084_


In [3]:
import torch

# CUDA 버전 확인
cuda_version = torch.version.cuda
print(f"CUDA Version: {cuda_version}")

# CUDA 연결 확인
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    # GPU 이름 확인
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Name: {gpu_name}")
else:
    print("CUDA is not available. Please check your CUDA installation.")

CUDA Version: 12.1
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4060 Ti
